In [14]:
# imports

import chromadb
from chromadb.utils import embedding_functions

In [15]:
## Terminal command to start a local host server
## chroma run --path "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data"

## instantiate chroma client
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [16]:
## access 2013 collection
collection_2013 = chroma_client.get_collection('2013pubmed')

In [17]:
from sentence_transformers import SentenceTransformer, util

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)

# User's question
user_question = 'CASK Disorders'

# Embed the user's question
user_question_embedding = sentence_transformer_ef([user_question])[0]

search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)
print(user_question)
# Print the search results
for i, result in enumerate(search_results['ids'][0]):
    document_id = result
    metadata = search_results['metadatas'][0][i]  # Access the corresponding metadata
    similarity_score = search_results['distances'][0][i]  # Access the corresponding similarity score
    document = search_results['documents'][0][i]  # Access the corresponding document

    print(f"PMID: {document_id}")
    # print(f"Title: {metadata['title']}")
    print(f"Author: {metadata['author']}")
    print(document)
    print(f"Similarity Score: {similarity_score}")
    print("---------------")


CASK Disorders
PMID: 24278995
Author: Ute Moog
CASK Disorders<SEP>Clinical characteristics: CASK disorders include a spectrum of phenotypes in both females and males. Two main types of clinical presentation are seen: Microcephaly with pontine and cerebellar hypoplasia (MICPCH), generally associated with pathogenic loss-of-function variants in CASKX-linked intellectual disability (XLID) with or without nystagmus, generally associated with hypomorphic CASK pathogenic variants MICPCH is typically seen in females with moderate-to-severe intellectual disability, progressive microcephaly with or without ophthalmologic anomalies, and sensorineural hearing loss. Most are able to sit independently; 20%-25% attain the ability to walk; language is nearly absent in most. Neurologic features may include axial hypotonia, hypertonia/spasticity of the extremities, and dystonia or other movement disorders. Nearly 40% have seizures by age ten years. Behaviors may include sleep disturbances, hand stereot

In [18]:
chroma_client.list_collections()

[Collection(name=2014pubmed),
 Collection(name=2015pubmed),
 Collection(name=2016-17pubmed),
 Collection(name=2013pubmed)]